Notebook for testing the general noisy system

## PSR

In [262]:
from qibochem.measurement.result import expectation_from_samples


def evaluate(circuit, hamiltonian, nshots):
    return expectation_from_samples(circuit, hamiltonian, nshots, group_pauli_terms="qwc")

In [263]:
def parameter_shift(
    circuit,
    hamiltonian,
    parameter_index,
    initial_state=None,
    scale_factor=1,
    nshots=None,
    nruns=1,
    repeated = False
):
    """In this method the parameter shift rule (PSR) is implemented.
    Given a circuit U and an observable H, the PSR allows to calculate the derivative
    of the expected value of H on the final state with respect to a variational
    parameter of the circuit.
    There is also the possibility of setting a scale factor. It is useful when a
    circuit's parameter is obtained by combination of a variational
    parameter and an external object, such as a training variable in a Quantum
    Machine Learning problem. For example, performing a re-uploading strategy
    to embed some data into a circuit, we apply to the quantum state rotations
    whose angles are in the form: theta' = theta * x, where theta is a variational
    parameter and x an input variable. The PSR allows to calculate the derivative
    with respect of theta' but, if we want to optimize a system with respect its
    variational parameters we need to "free" this procedure from the x depencency.
    If the `scale_factor` is not provided, it is set equal to one and doesn't
    affect the calculation.
    Args:
        circuit (:class:`qibo.models.circuit.Circuit`): custom quantum circuit.
        hamiltonian (:class:`qibo.hamiltonians.Hamiltonian`): target observable.
        parameter_index (int): the index which identifies the target parameter in the circuit.get_parameters() list
        initial_state ((2**nqubits) vector): initial state on which the circuit acts (default None).
        scale_factor (float): parameter scale factor (default None).
        repeated (bool): indicates if the gates that use the parameter have a translational symmetry
    Returns:
        np.float value of the derivative of the expectation value of the hamiltonian
        with respect to the target variational parameter.
    Example:
        .. testcode::
            import qibo
            import numpy as np
            from qibo import hamiltonians, gates
            from qibo.models import Circuit
            from qibo.derivative import parameter_shift
            # defining an observable
            def hamiltonian(nqubits = 1):
                m0 = (1/nqubits)*hamiltonians.Z(nqubits).matrix
                ham = hamiltonians.Hamiltonian(nqubits, m0)
                return ham
            # defining a dummy circuit
            def circuit(nqubits = 1):
                c = Circuit(nqubits = 1)
                c.add(gates.RY(q = 0, theta = 0))
                c.add(gates.RX(q = 0, theta = 0))
                c.add(gates.M(0))
                return c
            # initializing the circuit
            c = circuit(nqubits = 1)
            # some parameters
            test_params = np.random.randn(2)
            c.set_parameters(test_params)
            test_hamiltonian = hamiltonian()
            # running the psr with respect to the two parameters
            grad_0 = parameter_shift(circuit = c, hamiltonian = test_hamiltonian, parameter_index = 0)
            grad_1 = parameter_shift(circuit = c, hamiltonian = test_hamiltonian, parameter_index = 1)
    """
    """We modify the Qibo implementation to allow multiple parameter_index. For the case where a parameter is reused in different gates"""
    gradient = 0
    try:
        iter(parameter_index) # check if it is iterable
    except:
        parameter_index = [parameter_index]
    if repeated:
        nparam = len(parameter_index)
        parameter_index = parameter_index[:1]
    for parameter_index in parameter_index:
        # inheriting hamiltonian's backend
        backend = hamiltonian.backend

        # getting the gate's type
        gate = circuit.associate_gates_with_parameters()[parameter_index]

        # getting the generator_eigenvalue
        ###try: generator_eigenval = gate.generator_eigenvalue()
        ###except: 
        #generator_eigenval = 0.5   #### CAN'T DO SOMETHING LIKE THIS... ERROR GETS LOGGED ANYWAYS BY QIBO FUNCTION RAISE_ERROR, EVEN IF CAUGHT BY TRY
        # getting the generator_eigenvalue
        generator_eigenval = gate.generator_eigenvalue()

        # defining the shift according to the psr
        s = np.pi / (4 * generator_eigenval)

        # saving original parameters and making a copy
        original = np.asarray(circuit.get_parameters()).copy()
        shifted = original.copy()

        # forward shift and evaluation
        shifted[parameter_index] += s
        circuit.set_parameters(shifted)

        forward = 0
        backward = 0


        if nshots == None:
            forward = hamiltonian.expectation(
                backend.execute_circuit(circuit=circuit, initial_state=initial_state).state()
            )

            # backward shift and evaluation
            shifted[parameter_index] -= 2 * s
            circuit.set_parameters(shifted)

            backward = hamiltonian.expectation(
                backend.execute_circuit(circuit=circuit, initial_state=initial_state).state()
            )

        else:
            
            copied = shifted.copy()

            for _ in range(nruns):

                forward += evaluate(circuit, hamiltonian, nshots) #np.float64(hamiltonian.expectation_from_circuit(circuit, nshots=nshots).real)
                """backend.execute_circuit(
                    circuit=circuit, 
                    initial_state=initial_state, 
                    nshots=nshots
                    ).expectation_from_samples(hamiltonian)"""

                # backward shift and evaluation
                shifted[parameter_index] -= 2 * s
                circuit.set_parameters(shifted)

                backward += evaluate(circuit, hamiltonian, nshots)
                """backend.execute_circuit(
                    circuit=circuit, 
                    initial_state=initial_state, 
                    nshots=nshots
                    ).expectation_from_samples(hamiltonian)  """

                # restoring the original circuit
                shifted = copied.copy()
                circuit.set_parameters(copied)

            forward /= nruns
            backward /= nruns
                
        circuit.set_parameters(original)
        
        gradient += generator_eigenval * (forward - backward) * scale_factor

    if repeated: gradient *= nparam
    return gradient

## Non PSR

In [274]:
import qibo
from qibo import gates, Circuit
from qibo.symbols import I, X, Y, Z
import numpy as np

# Hamiltonian for the TFIsing model
def hamiltonianTFI(N, g):
    ham = I(0)*0
    for q in range(N):
        ham += Z(q)*Z((q+1)%N) + g * X(q)
    return qibo.hamiltonians.SymbolicHamiltonian(-ham)

qibo.gates.RZZ.generator_eigenvalue = lambda self: 0.5
def circ_inv(N, p):
    params = np.random.random((2*p,))*4*np.pi
    circuit_inv = Circuit(N)
    for q in range(N):
        circuit_inv.add(gates.H(q))  # WE START WITH |+> state
    for itp in range(p):
        for itzz in range(N):
            circuit_inv.add(gates.RZZ(itzz, (itzz+1)%N, params[2*itp]))
        for itx in range(N):
            circuit_inv.add(gates.RX(itx, params[2*itp + 1]))
    return circuit_inv, params

def circ_noinv(N, p):
    params2 = np.random.random((3*p,))*4*np.pi
    circuit_noinv = Circuit(N)
    for q in range(N):
        circuit_noinv.add(gates.H(q))
    for itp in range(p):
        for itzz in range(N):
            circuit_noinv.add(gates.RZZ(itzz, (itzz+1)%N, params2[3*itp]))
        for itx in range(N):
            circuit_noinv.add(gates.RX(itx, params2[3*itp + 1]))
        for ity in range(N):
            circuit_noinv.add(gates.RY(ity, params2[3*itp + 2]))
    return circuit_noinv, params2
    
def circs_shots_noisy_jac(N, p, g = 1, custom_operator=None, nshots=400, noise_map=None, epsilon = 0.02):
    if noise_map is None:
        noise_map = {i: list(zip(["X", "Z"], [epsilon, epsilon])) for i in range(N)}

    if custom_operator is None: 
        hamiltonian = hamiltonianTFI(N, g)
    else:
        hamiltonian = custom_operator
    
    circuit_inv, params = circ_inv(N,p)
    circuit_noinv, params2 = circ_noinv(N,p)

    circuit_inv = circuit_inv.with_pauli_noise(noise_map)
    circuit_noinv = circuit_noinv.with_pauli_noise(noise_map)

    def loss(params):
        circuit_inv.set_parameters(np.repeat(params,N))
        return evaluate(circuit_inv, hamiltonian, nshots)#np.float64(hamiltonian.expectation_from_circuit(circuit_inv, nshots=nshots).real)

    def loss2(params):
        circuit_noinv.set_parameters(np.repeat(params,N))
        return evaluate(circuit_noinv, hamiltonian, nshots)#np.float64(hamiltonian.expectation_from_circuit(circuit_noinv, nshots=nshots).real)

    gate_dependence_inv = {i: list(range(i*N, (i+1)*N)) for i in range(0,2*p)}
    gate_dependence_noinv = {i: list(range(i*N, (i+1)*N)) for i in range(0,3*p)}
    def jac_psr_inv(x, *args):
        circuit_inv.set_parameters(np.repeat(x,N))
        return np.array([parameter_shift(circuit_inv, hamiltonian, gate_dependence_inv[param], nshots=nshots, nruns=1, repeated=True) for param in range(len(params))])
    
    def jac_psr_noinv(x, *args):
        circuit_noinv.set_parameters(np.repeat(x,N))
        return np.array([parameter_shift(circuit_noinv, hamiltonian, gate_dependence_noinv[param], nshots=nshots, nruns=1, repeated=True) for param in range(len(params2))])
    
    return circuit_inv, circuit_noinv, params, params2, loss, loss2, jac_psr_inv, jac_psr_noinv


## With noise, some optimizers don't behave as well. For instance BFGS

# We tried implementing a noise resiliant alternative but it does not seem to work too well, because it stops convergence up to random error
# But we know we can keep running Gradient Descent, and stochastically it will converge


# We implement here a simple gradient descent, with a simple learning rate hyperparameter
def optimizer_gds(parameters, loss, gradient, Nepochs=250, lr=1e-2, epochs_print = 50, hyper={}, return_gradients=False):
    loss_list = np.empty((Nepochs,))
    if return_gradients: grad_list = np.empty((Nepochs,len(parameters)))
    for epoch in range(Nepochs):
        grad_val = gradient(parameters)
        parameters = parameters - lr * grad_val
        
        loss_val = loss(parameters)
        if not(epochs_print is None):
            if epoch % epochs_print == 0 or epoch == Nepochs - 1:
                print(f"Epoch {epoch}, Loss: {loss_val}")

        loss_list[epoch] = loss_val
        if return_gradients: grad_list[epoch] = grad_val
    
    if return_gradients: return parameters, loss_list, grad_list
    return parameters, loss_list

In [276]:
N = 6
nshots = 5
Nepochs = 25
p = 3
lr = 0.4e-2

epsilon = 0.02



data = []
for epsilon in [0.0001, 0.001, 0.01, 0.1]:
    for nshots in [40, 200, 400]:
        for rep in range(3):
            circuit_inv, circuit_noinv, params, params2, loss, loss2, jac1, jac2 = circs_shots_noisy_jac(N, p, nshots=nshots)
            _, loss_list, grads = optimizer_gds(params, loss, jac1, Nepochs = Nepochs, epochs_print=None, lr=lr, return_gradients=True)
            data.append({"nshots": nshots, "epsilon":epsilon, "loss": loss_list, "gradients": grads, "rep": rep})


SystemError: CPUDispatcher(<function apply_gate_kernel at 0x1383aa560>) returned a result with an exception set

In [277]:
print(data)

[{'nshots': 40, 'epsilon': 0.0001, 'loss': array([-1.50000000e+00, -1.80000000e+00, -1.65000000e+00, -8.00000000e-01,
       -1.60000000e+00,  1.11022302e-16, -1.05000000e+00, -1.35000000e+00,
       -1.85000000e+00, -2.45000000e+00, -1.35000000e+00, -2.30000000e+00,
       -1.10000000e+00, -2.35000000e+00, -1.70000000e+00, -9.50000000e-01,
       -2.40000000e+00, -1.45000000e+00, -1.40000000e+00, -1.20000000e+00,
       -1.95000000e+00, -2.80000000e+00, -2.30000000e+00, -1.85000000e+00,
       -2.05000000e+00]), 'gradients': array([[ 7.50000000e-01, -1.20000000e+00, -3.00000000e-01,
         1.80000000e+00,  1.65000000e+00, -3.75000000e+00],
       [ 2.85000000e+00,  1.95000000e+00, -7.50000000e-01,
        -9.00000000e-01,  6.90000000e+00,  1.80000000e+00],
       [ 1.50000000e+00,  5.10000000e+00,  3.75000000e+00,
         3.30000000e+00,  3.30000000e+00,  4.35000000e+00],
       [ 2.70000000e+00,  1.05000000e+00, -3.00000000e-01,
        -2.85000000e+00,  2.10000000e+00,  0.0000000

In [71]:
grads

array([[-6.  ,  3.12,  0.24,  1.44],
       [-5.52, -1.92,  2.88, -1.2 ],
       [-2.4 ,  2.16,  0.24, -3.84],
       [-2.16, -0.72,  3.36,  4.56]])

In [287]:
%matplotlib macosx
import matplotlib.pyplot as plt

for el in data:
    nshots = el["nshots"]
    rep = el["rep"]
    epsilon = el["epsilon"]
    epoch_range = np.arange(Nepochs)
    error = np.sqrt(2*N/nshots)
    loss_list = el["loss"]
    plt.plot(epoch_range, loss_list)
    plt.fill_between(epoch_range, loss_list+error, loss_list-error, alpha=0.1)
    plt.plot([0,Nepochs-1],[min(loss_list)]*2, '--')
    plt.plot([0,Nepochs-1],[-12.784906442999327]*2, '-.')
    plt.show()
    plt.pause(10)
    plt.close()
    

    

KeyboardInterrupt: 

## TESTING

In [288]:
import qibo
circ = qibo.Circuit(3)
circ.add(qibo.gates.X(0))

circ2 = qibo.Circuit(3)
circ2.add(qibo.gates.H(0))

circ += circ2

circ.draw()

0: ─X─H─
1: ─────
2: ─────


In [289]:
from qibo import Circuit, gates
from typing import List
import math

In [424]:
def dicke_state(nqubits: int, k: int, all_to_all_connectivity: bool = False, **kwargs):
    """Generate an :math:`n`-qubit Dicke state defined as

    .. math::
        \\ket{D^n_k} = \\frac{1}{{n\choose k}} \\sum_{\\ket{b}\\in B} \\ket{b}

    where :math:`B={\\ket{b}:b\\in {0,1}}^{\\otimes n} \\text{and } |b|=k}` is the set of
    bitstrings with fixed Hamming weight k (i.e. number of non-zero bits).
    Implemented following Lemma 2 from `arXiv:1904.07358 <https://arxiv.org/abs/1904.07358>`_ with 
    :math:`O(kn)` two-qubit gates and depth :math:`O(n)`. Additionally, if all-to-all connectivity is
    allowed, the improved version from `arXiv:2207.09998 <https://arxiv.org/abs/2207.09998>`_ with depth 
    :math:`O(k\\log(\\frac{n}{k}))` is used instead.

    Args:
        nqubits (int): number of qubits :math:`n >= 2`.
        k (int): bitstring's Hamming weight k.
        all_to_all_connectivity (bool): Decides which implementation to use.
        kwargs (dict, optional): additional arguments used to initialize a Circuit object.
            For details, see the documentation of :class:`qibo.models.circuit.Circuit`.

    Returns:
        :class:`qibo.models.circuit.Circuit`: Circuit that prepares the Dicke state.
    """
    circuit = Circuit(nqubits, **kwargs)
    if k == nqubits:
        circuit.add(gates.X(q) for q in range(nqubits))
        return circuit
    if all_to_all_connectivity:
        ## confusing because big endian interpretation in the paper for Unk is reversed from paper for all-to-all
        ## Circuit drawings are inverted with respect to paper for us

        # 1. We prepare disjoint sets of qubits / trees. Tier in ascending order
        disjoint_sets = [{"qubits": list(range(k*it, k*(it+1))), "tier": k, "children":[]} for it in range(nqubits//k)]
        nmodk = nqubits%k
        if nmodk != 0: disjoint_sets.append({"qubits": list(range(nqubits-nmodk, nqubits)), "tier": nmodk, "children":[]})
        disjoint_sets = list(reversed(disjoint_sets))  # we reverse to have in ascending order of tier

        trees = disjoint_sets.copy()
        # combine lowest tier trees into one tree. Could do binary search but isn't bottleneck
        while len(trees) > 1:
            trees[1]["tier"] += trees[0]["tier"]
            trees[1]["children"].append(trees[0])
            root = trees[1]
            del trees[1]
            del trees[0]
            trees.insert(sum(x["tier"] < root["tier"] for x in trees), root)

        # We initialize [1]xk  bitstring we distribute with WBD
        circuit.add(gates.X(q) for q in trees[0]["qubits"][-k:])

        # undo the union-by-tier algorithm applying WDB gates with root x and highest tier child y (y<x by construct.)
        # ugly code
        it = 0
        while it < len(trees):
            node = trees[it]
            if len(node["children"]) > 0:
                y = max(node["children"], key=lambda x: x["tier"])

                circuit += _dicke_WBD(nqubits, None, node["tier"], y["tier"], k, second_register=node["qubits"], first_register=y["qubits"], **kwargs)

                node["tier"] -= y["tier"]
                node["children"].remove(y)
                trees.append(y)
                if len(node["children"]) == 0: it+=1
            else: it+=1

        for node in disjoint_sets:
            circuit += _dicke_u(nqubits, node["qubits"], min(k, len(node["qubits"])), **kwargs)
                

    else:
        circuit.add(gates.X(q) for q in range(nqubits-k, nqubits))
        circuit += _dicke_u(nqubits, range(nqubits), k, **kwargs)

    return circuit

def _dicke_u(nqubits: int, qubits: List[int], k: int, **kwargs):
    """Implements :math:`U_{n,k}` gate from `arXiv:1904.07358 <https://arxiv.org/abs/1904.07358>`_"""
    circuit = Circuit(nqubits, **kwargs)
    n = len(qubits)
    if(k == 0):
        # State already in dicke state (as there is only 1 bitstring with hamming weight k=0 and k=n)
        return circuit
    for l in range(n, k, -1):
        circuit += _dicke_scs(nqubits, qubits[:l], k, **kwargs)
    for l in range(k, 1, -1):
        circuit += _dicke_scs(nqubits, qubits[:l], l-1, **kwargs)
    return circuit

def _dicke_scs(nqubits: int, qubits: List[int], k: int, **kwargs):
    """Implements :math:`SCS_{n,k}` gate from `arXiv:1904.07358 <https://arxiv.org/abs/1904.07358>`_"""
    n = len(qubits)
    circuit = Circuit(nqubits, **kwargs)
    circuit.add(gates.CNOT(qubits[n-2], qubits[n-1]))
    circuit.add(gates.RY(qubits[n-2], 2*math.acos(math.sqrt(1/n))).controlled_by(qubits[n-1]))
    circuit.add(gates.CNOT(qubits[n-2],qubits[n-1]))
    for l in range(2, k+1):
        circuit.add(gates.CNOT(qubits[n-1-l], qubits[n-1]))
        circuit.add(gates.RY(qubits[n-1-l], 2*math.acos(math.sqrt(l/n))).controlled_by(qubits[n-1],qubits[n-l]))
        circuit.add(gates.CNOT(qubits[n-1-l],qubits[n-1]))
    return circuit


def _dicke_WBD(nqubits: int, qubits: List[int], n: int, m: int, k: int, first_register: List[int]=None, second_register: List[int]=None, **kwargs):
    """Implements :math:`WBD^{n,m}_k` gate from `arXiv:2207.09998 <https://arxiv.org/abs/2207.09998>`_
    Only acts relevantly on last k digits of first m digits and last k digits of all n digits.
    These relevant first and second registers can be passed directly"""
    ## A bit confusing because they changed big endian interpretation in the paper!!!
    # we asume n-m > m (m <= n/2)
    if m > n/2: raise(ValueError("m must not be greater than n-m"))

    circuit = Circuit(nqubits, **kwargs)

    # IMPORTANT REMINDER, our first register and second register are reverse as paper,
    # Our drawn circuits are mirrored to circuit.
    # if m>k, m is truncated. Operations involving the most significant k-m digits can be removed
    if (first_register is None) and (second_register is None):
        first_register = qubits[max(m-k,0):m] 
        second_register = qubits[n-k:n]

    # (1) Switching from unary encoding to one hot encoding
    circuit.add(gates.CNOT(q, q+1) for q in reversed(second_register[:-1]))

    # (2) Adding a supperposition of hamming weights into the second register
    # this can be optimized
    # We follow Figure 4, but adjust definition of xi and si (suffix sum) to match 
    for l in reversed(range(1,k+1)):
        x = [math.comb(m,i)*math.comb(n-m,l-i) for i in range(l)]
        s = math.comb(n,l)
        circuit.add(gates.RY(first_register[-1], 2*math.acos(math.sqrt(x[0]/s))).controlled_by(second_register[-l]))
        s -= x[0]
        for q in range(1, min(l,m)):
            circuit.add(gates.RY(first_register[-q-1], 2*math.acos(math.sqrt(x[q]/s))).controlled_by(second_register[-l], first_register[-q]))
            s -= x[q]
    
    # (3) Go back to unary encoding, undoing one hot encoding
    circuit.add(gates.CNOT(q, q+1) for q in second_register[:-1])

    # (4) Substracting weight i in the first register from weight l in the second register. 
    # We use fredkin, controlled swap (can be decomposed into CNOT and Toffoli)
    fredkin = lambda control, s1, s2: (gates.CNOT(s2, s1), gates.TOFFOLI(control, s1, s2), gates.CNOT(s2,s1))

    dif = max(0, k-m)
    for control in range(dif, k):
        for q in range(control):
            circuit.add(fredkin(first_register[control-dif], second_register[control-q], second_register[control-q-1]))
        circuit.add(gates.CNOT(first_register[control-dif], second_register[0]))
    
    return circuit



In [416]:
circ = dicke_state(4,1)

circ.draw()

print(circ())

0: ─────────────────o─RY─o─
1: ──────────o─RY─o─X─o──X─
2: ───o─RY─o─X─o──X────────
3: ─X─X─o──X───────────────
(0.5+0j)|0001> + (0.5+0j)|0010> + (0.5+0j)|0100> + (0.5+0j)|1000>


In [417]:
### TEST FOR CHECKING WBD44
nq = 8
circ = Circuit(nq)
q = range(nq)

_dicke_WBD(nq, q, nq, 4, 4).draw()

0:     ────────────────RY─────────────────────────o────────────────────────── ...
1:     ─────────────RY─o────────RY────────────────|───o───o────────────────── ...
2:     ──────────RY─o──|─────RY─o─────RY──────────|───|───|───o─────o───o──── ...
3:     ───────RY─o──|──|──RY─o──|──RY─o──RY───────|───|───|───|─────|───|───o ...
4:     ─────o─o──o──o──o──|──|──|──|──|──|──o─────X─o─X─o─X───|───o─X─o─X───| ...
5:     ───o─X─────────────o──o──o──|──|──|──X─o─────X─o─X───o─X─o─X─o─X─────| ...
6:     ─o─X────────────────────────o──o──|────X─o───────────X─o─X─────────o─X ...
7:     ─X────────────────────────────────o──────X─────────────────────────X─o ...

0: ... ─────────────────
1: ... ─────────────────
2: ... ─────────────────
3: ... ─────o─────o───o─
4: ... ─────|───o─X─o─X─
5: ... ───o─X─o─X─o─X───
6: ... ─o─X─o─X─────────
7: ... ─X───────────────


In [423]:
### TEST FOR VALIDATING WBD/U BY REPLICATING QUIRK CIRCUIT AFTER DEFINITION 2 (IMPLEMENT FIGURE 2)
print("MANUAL WBD CONSTRUCTION TEST: ")
nqubits = 11
circ = Circuit(nqubits)
qubits = range(nqubits)

# flip last k bits, to generate Dnk
k = 3
circ.add( gates.X(nqubits-1-q) for q in range(k) )   # can test for l=0, 1, 2 or 3

circ += _dicke_WBD(nqubits, qubits, 11, 5, 3)

circ += _dicke_WBD(nqubits, qubits[-6:], 6, 3, 3)
circ += _dicke_WBD(nqubits, qubits[:5], 5, 2, 3)

circ += _dicke_u(nqubits, qubits[:2], 2)
circ += _dicke_u(nqubits, qubits[2:5], 3)
circ += _dicke_u(nqubits, qubits[5:8], 3)
circ += _dicke_u(nqubits, qubits[8:], 3)

circ.draw()
print(circ())


### WE ALSO VALIDATE IT GIVES THE SAME AS WITHOUT FULL_CONENCTIVITY
print("\n\nNON-FULL CONNECTIVITY SANITY CHECK: ")
print(dicke_state(nqubits, k)())


### AND FINALLY TEST WE ALSO GET THIS CIRCUIT FROM THE ALL_TO_ALL_CONNECTIVITY
print("\n\nALL-TO-ALL CONNECTIVITY REPLICATE: ")
circ2 = dicke_state(nqubits, k, all_to_all_connectivity=True)
circ2.draw()
print(circ2())


MANUAL WBD CONSTRUCTION TEST: 
0 :     ────────────────────────────────────────────────────────────────────── ...
1 :     ────────────────────────────────────────────────────────────────────── ...
2 :     ─────────────RY──────────────o──────────────────────────────────────── ...
3 :     ──────────RY─o─────RY────────|───o───o──────────────────────────────── ...
4 :     ───────RY─o──|──RY─o──RY─────|───|───|───o─────o───o────────────────── ...
5 :     ───────|──|──|──|──|──|──────|───|───|───|─────|───|───────────RY───── ...
6 :     ───────|──|──|──|──|──|──────|───|───|───|─────|───|────────RY─o─────R ...
7 :     ───────|──|──|──|──|──|──────|───|───|───|─────|───|─────RY─o──|──RY─o ...
8 :     ─X───o─o──o──o──|──|──|──o───X─o─X─o─X───|───o─X─o─X───o─o──o──o──|──| ...
9 :     ─X─o─X──────────o──o──|──X─o───X─o─X───o─X─o─X─o─X───o─X──────────o──o ...
10:     ─X─X──────────────────o────X───────────X─o─X─────────X──────────────── ...

0 : ... ────────────────────────────────────────RY────R

In [376]:
import math
import numpy as np
x = lambda n, m, l: [math.comb(m,i)*math.comb(n-m,l-i) for i in range(l+1)]
s = lambda n, m, l: [sum(x(n,m,l)[i:]) for i in range(l)]

print(x(5,2,3))
print(s(5,2,3))

np.array([0,1,2,3])[-4]

[1, 6, 3, 0]
[10, 9, 3]


0

In [377]:
math.comb(6,3)

20

In [387]:
dicke_state(162, 18, all_to_all_connectivity=True)

In [311]:
l = [1, 2, 5, 7]

for x in l:
    print(x)
    if x == 2:
        l.m
        del x

l


1
2
5
7
3


[1, 2, 5, 7, 3]